add data

In [ ]:
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=4000)

word_index = tf.keras.datasets.imdb.get_word_index()
index2word = dict((i + 3, word) for (word, i) in word_index.items())
index2word[0] = '[pad]'
index2word[1] = '[bos]'
index2word[2] = '[oov]'
x_train = np.array([' '.join([index2word[idx] for idx in text]) for text in x_train])
x_test = np.array([' '.join([index2word[idx] for idx in text]) for text in x_test])

1654784/1641221 [==============================] - 0s 0us/step


In [ ]:
vocabulary = list()
for text in x_train:
  tokens = text.split()
  vocabulary.extend(tokens)

vocabulary = set(vocabulary)
print(len(vocabulary))

3998


In [ ]:
from tqdm import tqdm

x_train_binary = list()
x_test_binary = list()

for text in tqdm(x_train):
  tokens = text.split()
  binary_vector = list()
  for vocab_token in vocabulary:
    if vocab_token in tokens:
      binary_vector.append(1)
    else:
      binary_vector.append(0)
  x_train_binary.append(binary_vector)

x_train_binary = np.array(x_train_binary)

for text in tqdm(x_test):
  tokens = text.split()
  binary_vector = list()
  for vocab_token in vocabulary:
    if vocab_token in tokens:
      binary_vector.append(1)
    else:
      binary_vector.append(0)
  x_test_binary.append(binary_vector)

x_test_binary = np.array(x_test_binary)

100%|██████████| 25000/25000 [06:06<00:00, 68.13it/s]


In [ ]:
print(x_test_binary[2])
print(y_test[2])
print(x_train_binary[2])
print(y_train[2])
print('# idiotitwn(features):\n')
print('len(x_test_binary): ', len(x_test_binary))
print('len(y_test): ', len(y_test))

[1 0 0 ... 0 0 0]
1
[1 0 0 ... 0 0 0]
0
# idiotitwn(features):

len(x_test_binary):  25000
len(y_test):  25000


Calculate entropy

In [ ]:
import math
from math import log2 

def calculate_entropy_total_word(binary_array, c, word, word_position = -1):
  first = 0 # c value
  second = 0 # other than c value
  count = 0 # lengeth
  if word_position == -1:
    word_position = return_word(word) # the word column

  for element in binary_array:
      pos = element[word_position]
      count += 1
      if(pos == c):
        first += 1
      else:
        second += 1
  return -(first/count) * math.log2((first/count) - (second/count) * math.log2((second/count)))

def calculate_entropy_word(binary_array, c, word, word_position = -1):
  first = 0 # c value
  second = 0 # other than c value
  count = 0 # lengeth

  if word_position == -1:
    word_position = return_word(word) # the word column
  
  for element in binary_array:
      element = element[word_position]
      count += 1
      if(element == c):
        first += 1
      else:
        second += 1
  
  val = -(first/count) * log2((first/count) - (second/count) * log2((second/count)))
  return val


def calculate_entropy_total(binary_array):
  first = 0 # c value
  second = 0 # other than c value
  count = 0 # lengeth
  for element in binary_array:
      for pos in element:
        count += 1
        if(pos == 1):
          first += 1
        else:
          second += 1
  return -(first/count) * log2((first/count) - (second/count) * log2((second/count)))


  

print(calculate_entropy_total(x_train_binary))

0.1133869708654645


return the position of a word

In [ ]:
# -1 = has not found the word
def return_word(word):
  count = 0
  for element in vocabulary:
    if word == element:
      return count
    else:
      count += 1
  
  return -1

calculate probability

In [ ]:
def calculate_probability_old(array, c, word, word_position = -1):
  count = 0
  if word_position == -1:
    word_position = return_word(word) # the word column
  for element in array:
    if(element[word_position] == c):
      count += 1
  return count / len(array)



def calculate_probability(array, c, word, word_position = -1):
  count = 0
  p = 0
  index = 0
  if word_position == -1:
    word_position = return_word(word) # the word column
  for element in array:
    if(element[word_position] == 1):
      count += 1
      if(y_train[index] == c):
        p += 1
    index += 1
  return p / count

calculate sum

In [ ]:
def calculate_sum(binary_array, word):
  try:
    word_position = word.index
    p0 = (calculate_probability_old(binary_array, 0, word, word_position))
    p1 = (calculate_probability_old(binary_array, 1, word, word_position))
    word.setEntropy0(calculate_entropy_word(binary_array, 0, word,  word_position))
    word.setEntropy1(calculate_entropy_word(binary_array, 1, word,  word_position))
    a = p0 * word.entropy0
    b = p1 * word.entropy1
    return  a + b
  except:
    return None
  

calculate IG

In [ ]:
def calculate_IG(binary_array, c, word, calculate_entrupy_total = 'no value'):
  if calculate_entrupy_total == 'no value':
    calculate_entrupy_total = calculate_entropy_total(binary_array)
  try:
    word.setIg(calculate_entrupy_total - calculate_sum(binary_array, word))
  except:
    word.setIg(None)

Get featured IG

We have some problem with 2 words


In [ ]:
import math
def get_featured_max_IG(binary_array, c):
  count = 0
  a = list()
  cet = calculate_entropy_total(binary_array)
  max = -9^999999999
  maxS = ""
  for x in vocabulary:
    count += 1
    try:
      result = calculate_IG(binary_array,1 , x, cet)
      if result > max:
        max = result
        maxS = x
      a.append(calculate_IG(binary_array,1 , x, cet))
    except:
      continue

  print("\n")
  print("\n")
  print("\n")
  return (result, maxS)

print(get_featured_max_IG(x_train_binary, 1))

UnboundLocalError: ignored

Node

In [ ]:
class Node:
  value: int
  p0: int
  p1: int
  left = None # must be Node
  right = None # must be Node

  def __init__(self, v: int = None):
    self.value = v
  
  def setLeftChild(self, v):
    self.left = v

  def setRightChild(self, v):
    self.right  = v

  def setChilds(self, l, r):
    self.setLeftChild(l)
    self.setRightChild(r)
  def result(self):
    if(p0 > p1):
      return "positive"
    return "negative"
  def __str__(self):
    return str(self.value)
      


h = Node(5)
h.setLeftChild(Node(3))
print(h)


    #create else

5


Remove words

In [ ]:
def hasTheWord(word1: Word, word2: Word):
  index1 = word1.index
  index2 = word2.index
  for x in x_train_binary:
    if(x[index1] == 1 and x[index2] == 1):
      return 1

  return 0


In [ ]:
class Word:

  def __init__(self, value:str, index:int):
    self.value = value
    self.index = index
    self.p0 = None
    self.p1 = None
    self.ig = None
    self.entropy0 = None
    self.entropy1 = None


  def setP0(self, p0):
    self.p0 = p0
  
  def setP1(self, p1):
    self.p1 = p1
  
  def setIg(self, ig):
    if(ig != None):
      self.ig = float(ig)

  def setEntropy0(self, entropy0):
    self.entropy0 = entropy0

  def setEntropy1(self, entropy1):
    self.entropy1  = entropy1

  def __str__(self):
    return self.ig

  def __repr__(self):
    return self.value + " " + str(self.ig)
  

In [ ]:
#there is the word in the setence
def findValueFromArray(array, index):
  if len(array) < index:
    return None
  count = 0
  for x in array:
    if(count == index):
      return x
    count+= 1

In [ ]:
def findTheNextWord(maxWord, array, c):
  nextWord = array.pop()
  while hasTheWord(maxWord, nextWord) != c:
    if len(array) == 0:
      return None
    try:
      nextWord = array.pop()
    except:
      return None
  return nextWord


Main run one

In [ ]:
test = x_train_binary[0]
print(y_train[0])
ar = x_train_binary.tolist()
i = 0
et = calculate_entropy_total(x_train_binary)
words = list()
for x in vocabulary:
  words.append(Word(x, i))
  i+= 1
words_to_remove = list()
for word in words:
  calculate_IG(x_train_binary, 1, word, et)
  word.setP0(calculate_probability(x_train_binary, 0, word, word.index))
  word.setP1(calculate_probability(x_train_binary, 1, word, word.index))
  if(word.ig == None):
    words_to_remove.append(word)



for x in words_to_remove:
  words.remove(x)



words.sort(key=lambda x: x.ig, reverse=True)
max = words.pop()
stopTheWhile = False
#next = findTheNextWord(max, words, c)
while len(words) !=0 and not stopTheWhile:
  c = findValueFromArray(test, max.index)
  while len(words) != 0 and max.p0 != 1 and max.p1 !=1:
    next = findTheNextWord(max, words, c)
    if(next == None):
      stopTheWhile = True
      break
    max = next


if(max.p0 > max.p1):
  print("bad")
  print(max.p0)
  print(max.p1)
else:
  print("good")
  print(max.p0)
  print(max.p1)



1
bad
0.6733966745843231
0.32660332541567694


create words

In [ ]:
test = x_train_binary[0]
ar = x_train_binary.tolist()
i = 0
et = calculate_entropy_total(x_train_binary)
words = list()
for x in vocabulary:
  words.append(Word(x, i))
  i+= 1
words_to_remove = list()
for word in words:
  calculate_IG(x_train_binary, 1, word, et)
  word.setP0(calculate_probability(x_train_binary, 0, word, word.index))
  word.setP1(calculate_probability(x_train_binary, 1, word, word.index))
  if(word.ig == None):
    words_to_remove.append(word)



for x in words_to_remove:
  words.remove(x)



words.sort(key=lambda x: x.ig, reverse=True)

In [ ]:
print(len(vocabulary))
print(vocabulary)

3998
{'one', 'learn', 'existent', 'food', 'portrayal', 'challenge', 'biggest', 'certain', 'much', 'sets', 'gone', 'inspector', 'general', 'seeing', 'destroyed', 'reach', 'planning', 'wondering', 'unbelievably', 'enemy', 'lord', 'boys', 'everyday', 'passed', 'sadly', 'grace', 'wars', 'focus', 'comic', 'knowledge', 'fi', 'moving', 'support', 'appearance', 'previously', 'power', 'qualities', 'piece', 'learning', 'appalling', 'prefer', 'personalities', 'starring', 'attack', 'content', 'junk', 'leave', 'bill', 'placed', 'future', 'whose', 'supposedly', 'loves', 'projects', 'superior', 'wants', 'women', 'female', 'corny', 'moves', 'contains', 'falls', 'follows', 'writer', 'join', 'mild', 'watch', 'setting', 'canada', 'ned', 'decides', 'besides', 'steals', 'vicious', 'altogether', 'higher', 'meaning', 'dreams', 'land', 'lacks', 'cartoon', 'meaningful', '2003', 'standard', 'whether', 'lazy', 'offer', 'suffers', 'produce', 'astaire', 'shows', 'lit', 'stunts', 'estate', "wouldn't", 'cops', '99',

Run it all

In [ ]:
j = 0
#next = findTheNextWord(max, words, c)
good_result = 0
print(len(x_train_binary))
print("hello let's start")
print(words)
for sentense in x_train_binary:
  copy_words = words.copy()
  max = copy_words.pop()
  stopTheWhile = False
  while len(copy_words) !=0 and not stopTheWhile:
    c = findValueFromArray(sentense, max.index)
    while len(copy_words) != 0 and max.p0 != 1 and max.p1 !=1:
      next = findTheNextWord(max, copy_words, c)
      if(next == None):
        stopTheWhile = True
        break
      max = next
  if(max.p0 > max.p1):
    if 0 == y_train[j]:
      good_result += 1
  else:
    if 1 == y_train[j]:
      good_result += 1

  j+=1
  print(good_result/j*100)



print("teliko")
print(good_result/j*100)


25000
hello let's start
[script 0.3366060565807609, making 0.33659591775813963, since 0.3365957909857916, far 0.3365943694419989, anything 0.3365918453771541, bit 0.33658169057053244, feel 0.3365800229961272, is 0.336571646998787, last 0.33656189135622294, isn't 0.336548249801569, role 0.3365427898808765, probably 0.3365064280801708, gets 0.33648882256109003, interesting 0.3364760748189493, point 0.3364681180103855, might 0.3364673777563673, away 0.3364313880200584, whole 0.3364186880303411, minutes 0.3363875914987499, done 0.33637594525363157, almost 0.33636193821255256, sure 0.33630616999331064, may 0.33626951913374364, hard 0.33626719918047443, always 0.33625272971172193, this 0.3362444835448672, there's 0.33615636113620234, young 0.3361533863585179, original 0.3361354210765823, comedy 0.3361247649397312, least 0.3360993077263084, both 0.33602087954999316, between 0.33596561502493005, must 0.33593709496332697, i've 0.3359020932516833, action 0.33589848477429557, own 0.33589617737522

KeyboardInterrupt: ignored

ID3 class

In [ ]:
class ID3:
    def __init__(self, x_tr_bin:list(), y_tr:list(), wor:list()):
        self.x_tr_bin = x_tr_bin
        self.y_tr = y_tr
        self.wor  = wor

    def run(self):
      j = 0 #next = findTheNextWord(max, words, c)
      good_result = 0
      print('------------------------------stolou------- len(self.x_tr_bin)', len(self.x_tr_bin))
      print("------------------------------stolou------- hello let's start")
      print(self.wor)
      for sentense in self.x_tr_bin:
        print('!!@@@@!!@!@!@protash')
        copy_words = self.wor.copy()
        max = copy_words.pop()
        stopTheWhile = False
        while len(copy_words) !=0 and not stopTheWhile:
          c = findValueFromArray(sentense, max.index)
          while len(copy_words) != 0 and max.p0 != 1 and max.p1 !=1:
            next = findTheNextWord(max, copy_words, c)
            if(next == None):
              stopTheWhile = True
              break
            max = next
        if(max.p0 > max.p1):
          if 0 == self.y_tr[j]:
            good_result += 1
        else:
          if 1 == self.y_tr[j]:
            good_result += 1

        j+=1
        
        print('------------------------------stolou------- ', good_result/j*100)
      return good_result/j*100

Run ID3 class

In [ ]:
id3Obj = ID3(x_train_binary, y_train, words)
id3Obj.run()

In [ ]:
import random
from random import randint
"""
- emina sth fash pou enna mikratw thn lista words(tixeo iposinolo, idio gia oulla)
na dw dld pws enna gnt o xorismos giati en j h lista words pou enna allassei gia kathe dentro
na eshei omos diplotipa
"""
#print(len(words))
#print(words)

numOfTrees = 3 # number of trees in random forest, kanonika prepei na en tosa nmz: int(len(x_train_binary) * 0.3)

# random size for words' subset(prepei na einai idio gia kathe dentro)
wordsSize = 100 # randint(100, len(words))
newl = random.choices(words, k=wordsSize) # subets with same length
#print(len(newl))

"""
id3Obj = ID3(x_train_binary, y_train, words)
"""
randomForest = []
resRF = []
newX_train_binary = x_train_binary[:5] # prosorina, meta enna mpei oullh alios osh theloume
newY_train = y_train[:5] # >>

for i in range(numOfTrees):
  subWords = random.choices(words, k=5) # subWords = random.choices(words, k=wordsSize)
  randomForest.append(ID3(newX_train_binary, newY_train, subWords))

print(len(randomForest))

for obj in randomForest:
  temp = obj.run()
  resRF.append(temp)
  print('end of tree\n\n\n\n', temp)



print('\nresRF: ', resRF)


3
------------------------------stolou------- len(self.x_tr_bin) 5
------------------------------stolou------- hello let's start
[albert 0.16984826261803518, opens 0.19048361656670298, aliens 0.16777358869801132, prom 0.1360683644001826, versus 0.1489112989694103]
!!@@@@!!@!@!@protash
------------------------------stolou-------  100.0
!!@@@@!!@!@!@protash
------------------------------stolou-------  50.0
!!@@@@!!@!@!@protash
------------------------------stolou-------  33.33333333333333
!!@@@@!!@!@!@protash
------------------------------stolou-------  50.0
!!@@@@!!@!@!@protash
------------------------------stolou-------  40.0
end of tree



 40.0
------------------------------stolou------- len(self.x_tr_bin) 5
------------------------------stolou------- hello let's start
[built 0.18339672335546067, humorous 0.19005296181525247, importance 0.16102460088545462, decade 0.18632747674102956, branagh 0.1450587110408048]
!!@@@@!!@!@!@protash
------------------------------stolou-------  100.0
